In [35]:
import pandas as pd

file = '../data/01_raw/2674.xls'

# Dicionário com o range de linhas de cada continente no arquivo
continent_range = {
    'África do Sul': '11:9',
    'América Central e Caribe': '21:11',
    'América do Norte': '33:3',
    'América do Sul': '37:12',
    'América do Sul': '37:12',
    'Ásia': '50:19',
    'Europa': '70:33',
    'Europa': '104:3',
    'Países não especificados': '107:1',
}


def get_continent_data(continent):
    """
    Retorna um DataFrame normalizado com os dados de um continente específico
    """
    start, end = continent_range[continent].split(':')
    df = pd.read_excel(file, '2019', skiprows=int(
        start), nrows=int(end),  usecols='A:D')
    df.columns = [continent, 'Total', 'Aérea', 'Marítima']

    # Troca '-' por 0 nas colunas Toal, Aérea e Marítima
    df['Total'] = df['Total'].replace('-', 0)
    df['Aérea'] = df['Aérea'].replace('-', 0)
    df['Marítima'] = df['Marítima'].replace('-', 0)
    
    # Parse as colunas para inteiros
    df['Total'] = df['Total'].astype(int)
    df['Aérea'] = df['Aérea'].astype(int)
    df['Marítima'] = df['Marítima'].astype(int)

    return df

get_continent_data('Países não especificados')

C:\Users\rafae\AppData\Local\Temp\ipykernel_14572\1167670728.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Marítima'] = df['Marítima'].replace('-', 0)


,Países não especificados,Total,Aérea,Marítima
0,Países não especificados,7,7,0
